In [1]:
import h5py
filenameCamera = '/media/is-lab/2E82556B82553913/SP/camera/2016-01-31--19-19-25.h5'
fCamera = h5py.File(filenameCamera, 'r')

list (fCamera)

# List all groups
#print("Keys: %s" % f.keys())
#a_group_key = list(f.keys())[0]

# Get the data
#data = list(f[a_group_key])
#print (data)

['X']

In [2]:
fCamera['X'].shape

(19731, 3, 160, 320)

In [3]:
import h5py
filenameLog = '/media/is-lab/2E82556B82553913/SP/log/2016-01-31--19-19-25.h5'
fLog = h5py.File(filenameLog, 'r')

list (fLog)


['UN_D_cam1_ptr',
 'UN_D_cam2_ptr',
 'UN_D_camphone_ptr',
 'UN_D_lidar_ptr',
 'UN_D_radar_msg',
 'UN_D_rawgps',
 'UN_T_cam1_ptr',
 'UN_T_cam2_ptr',
 'UN_T_camphone_ptr',
 'UN_T_lidar_ptr',
 'UN_T_radar_msg',
 'UN_T_rawgps',
 'blinker',
 'brake',
 'brake_computer',
 'brake_user',
 'cam1_ptr',
 'cam2_ptr',
 'camphone_ptr',
 'car_accel',
 'fiber_accel',
 'fiber_compass',
 'fiber_compass_x',
 'fiber_compass_y',
 'fiber_compass_z',
 'fiber_gyro',
 'fiber_temperature',
 'gas',
 'gear_choice',
 'gps_1_fix',
 'gps_1_pos',
 'gps_1_vel',
 'gps_2_fix',
 'gps_2_pos',
 'gps_2_vel',
 'idx',
 'imu_accel',
 'imu_compass',
 'imu_gyro',
 'rpm',
 'rpm_post_torque',
 'selfdrive',
 'speed',
 'speed_abs',
 'speed_fl',
 'speed_fr',
 'speed_rl',
 'speed_rr',
 'standstill',
 'steering_angle',
 'steering_torque',
 'times',
 'velodyne_gps',
 'velodyne_heading',
 'velodyne_imu']

In [4]:

a = fLog['cam1_ptr']
#len([i for i, e in enumerate(a) if e != 0])
a.shape

(98719,)

In [5]:
a1 = list(a)
#print (a1)
#print(a[3000:4000])
#a.index("621")
#print(a[294916])

In [6]:
a1.index(621)

3141

In [7]:
fLog['speed'][3148]

-0.7588629396396555

In [8]:
prev = 0
k = 0
sum = 0
l = 0
m = list()
for i in a:
    if i!=prev:
        m.append(sum/l)
        l = 1
        sum = fLog['speed'][k];
    else:
        sum = sum + fLog['speed'][k]
        l = l+1
    prev = i
    k = k+1
    
print(len(m))

19731


In [9]:
#len(m)

In [10]:
image = fCamera['X'][4090,:,:,:]
print(image.shape)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

def rgb2gray(rgb):
    r, g, b = rgb[0,:,:], rgb[1,:,:], rgb[2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    #return np.dot(rgb[:3,:,:], [0.299, 0.587, 0.114])
    return gray
     
from scipy.misc import toimage
from scipy.misc import imshow
#imshow(image)

gray = rgb2gray(image)  
#imshow(gray)

(3, 160, 320)


In [11]:
x = fCamera['X'].value
y = np.asarray(m)
#x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)


/home/is-lab/.local/lib/python3.5/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [12]:
c = np.c_[x.reshape(len(x), -1), y.reshape(len(y), -1)]
np.random.shuffle(c)

In [13]:
fCamera.close()
fLog.close()
x = c[:, :x.size//len(x)].reshape(x.shape)
y = c[:, x.size//len(y):].reshape(y.shape)
print(x.shape, y.shape)

(19731, 3, 160, 320) (19731,)


In [14]:
x_train = x[0:16000, :, :, :]
x_test = x[16000:19731, :, :, :]
y_train = y[0:16000]
y_test = y[16000:19731]
print (x_train.shape)
print(x_test.shape)
print(len(y_train))
print(len(y_test))

(16000, 3, 160, 320)
(3731, 3, 160, 320)
16000
3731


In [15]:
x_train = np.rollaxis(x_train, 1, 4)
x_test = np.rollaxis(x_test, 1, 4)
print(x_train.shape)
x_test.shape

(16000, 160, 320, 3)


(3731, 160, 320, 3)

In [16]:
#x_train = x_train  / 255.0
#x_test = x_test / 255.0
#memory error

In [17]:
import tensorflow as tf
def multitask_loss(y_true, y_pred):
    loss1 = tf.losses.mean_squared_error(y_true, y_pred)

    jointLoss = loss1
    return jointLoss
    

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Lambda, ELU
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers.convolutional import Convolution2D

batch_size = 64
num_classes = 5
epochs = 25

# input image dimensions
img_rows, img_cols = 160, 320
channels = 3
'''
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same',input_shape=(img_rows, img_cols, channels)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='normal'))
#model.add(Activation('sigmoid'))


model.compile(loss=multitask_loss,
              optimizer='adam')
'''

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape=(img_rows, img_cols, channels),
            output_shape=(img_rows, img_cols, channels)))
model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))

model.compile(optimizer="adam", loss="mse")



model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Using TensorFlow backend.
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (8, 8), strides=(4, 4), padding="same")`
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), strides=(2, 2), padding="same")`
/home/is-lab/.local/lib/python3.5/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), strides=(2, 2), padding="same")`


Train on 16000 samples, validate on 3731 samples
Epoch 1/25
16000/16000 [==============================] - 99s 6ms/step - loss: 65.5564 - val_loss: 36.5717
Epoch 2/25
16000/16000 [==============================] - 99s 6ms/step - loss: 30.6311 - val_loss: 35.7914
Epoch 3/25
16000/16000 [==============================] - 99s 6ms/step - loss: 21.1726 - val_loss: 17.0716
Epoch 4/25
16000/16000 [==============================] - 99s 6ms/step - loss: 15.0275 - val_loss: 28.4057
Epoch 5/25
16000/16000 [==============================] - 100s 6ms/step - loss: 12.0156 - val_loss: 59.7348
Epoch 6/25
16000/16000 [==============================] - 99s 6ms/step - loss: 9.2587 - val_loss: 54.7178
Epoch 7/25
16000/16000 [==============================] - 99s 6ms/step - loss: 8.0480 - val_loss: 46.1708
Epoch 8/25
16000/16000 [==============================] - 99s 6ms/step - loss: 7.4522 - val_loss: 58.8054
Epoch 9/25
16000/16000 [==============================] - 100s 6ms/step - loss: 6.3644 - val_loss

In [19]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model10epochsshuffledSpeedComma.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model10epochsshuffledSpeedComma.h5")
print("Saved model to disk")


Saved model to disk


In [20]:
model.evaluate(x_test, y_test)

3731/3731 [==============================] - 7s 2ms/step


57.11772309740118

In [21]:
y_predict = model.predict(x_test)

In [22]:
from sklearn.metrics import r2_score
print(r2_score(y_test, y_predict))

/home/is-lab/.local/lib/python3.5/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: [Errno 12] Cannot allocate memory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


0.4695701236086026


In [23]:
x = np.rollaxis(x, 1, 4)

y_predictWhole = model.predict(x)

In [24]:
print(r2_score(y, y_predictWhole))

0.46530682654498157
